In [1]:
from dotenv import load_dotenv
from pyflink.table.confluent import ConfluentSettings, ConfluentTools
from pyflink.table.expressions import *
from pyflink.table import TableEnvironment

In [2]:
from dotenv import load_dotenv
load_dotenv("./.env")

True

In [3]:
settings = ConfluentSettings.from_global_variables()
settings = settings.in_streaming_mode()
env = TableEnvironment.create(settings)

In [4]:
# settings = ConfluentSettings.new_builder() \
#   .set_cloud("gcp").set_region("us-west1") \
#   .set_flink_api_key("key").set_flink_api_secret("secret") \
#   .set_organization_id("org_id").set_environment_id("env_id") \
#   .set_compute_pool_id("pool_id").build()

In [5]:
env.list_catalogs() # list Kafka environments

['default', 'examples', 'msds682']

In [6]:
env.use_catalog("msds682")

In [7]:
env.list_databases() # list Kafka clusters

['INFORMATION_SCHEMA', 'msds682']

In [8]:
env.use_database("msds682")

In [9]:
env.list_tables() # list Kafka topics

['system.usage',
 'system.usage$errors',
 'system.usage.flink',
 'system.usage.flink$errors',
 'system.usage.new',
 'system.usage.new$errors',
 'system.usage.windowed',
 'system.usage.windowed$errors',
 'window_results',
 'window_results$errors']

In [ ]:
table = env.from_path("`system.usage`")

In [11]:
ConfluentTools.print_changelog_limit(table, 10)

+----+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+----------------------+-------------------------+
| op |                            key |                             id |                      cpu_usage |                      cpu_stats |                   memory_usage |            timestamp |                $rowtime |
+----+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+----------------------+-------------------------+
| +I |                          x'35' |                              5 |                           21.6 | [20257.0, 176754.0, 1.22123... |                           65.4 |        1763701684564 | 2025-11-20 13:08:04.830 |
| +I |                          x'34' |                              4 |                           21.9 | [25531.0, 

In [12]:
result = table.where(col("id")=='2').select(col("id"),col("memory_usage"), col("$rowtime").alias("rowtime"))

In [13]:
ConfluentTools.print_changelog_limit(result, 10)

+----+--------------------------------+--------------------------------+-------------------------+
| op |                             id |                   memory_usage |                 rowtime |
+----+--------------------------------+--------------------------------+-------------------------+
| +I |                              2 |                           65.6 | 2025-11-20 13:08:08.520 |
| +I |                              2 |                           66.0 | 2025-11-20 13:08:21.183 |
| +I |                              2 |                           65.8 | 2025-11-20 13:08:22.299 |
| +I |                              2 |                           64.6 | 2025-11-20 13:08:28.665 |
| +I |                              2 |                           64.4 | 2025-11-20 13:08:29.711 |
| +I |                              2 |                           64.5 | 2025-11-20 13:08:30.750 |
| +I |                              2 |                           64.7 | 2025-11-20 13:08:39.297 |
| +I |    